#### libs and imports
Run these beforehand!

In [1]:
#@title Mount drive and set current working directory { run: "auto", vertical-output: true }
users_drive_location = {
    "jr.gaglione@utexas.edu": "MyDrive/Research/davinci2automata/",
    "yunhaoyang234@utexas.edu": "MyDrive/davinci2automata/",
    # add your drive location
}

# mount drive
from google.colab import drive
mountpoint = '/content/drive'
drive.mount(mountpoint, force_remount=True)

# get email and preferred drive location
from google.colab import auth
auth.authenticate_user()
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
print(f"You are identified as: {gcloud_tokeninfo['email']!r}")
drive_location = users_drive_location.get(gcloud_tokeninfo['email'], "Shareddrives/davinci2automata/")
print(f"Your current *drive* directory will be set to {drive_location!r}")

# set curent working directory
import os,sys
cwd = os.path.join(mountpoint,drive_location)
if cwd not in sys.path: sys.path.append(cwd)
%cd {cwd}
!pwd

Mounted at /content/drive
You are identified as: 'yunhaoyang234@utexas.edu'
Your current *drive* directory will be set to 'MyDrive/davinci2automata/'
/content/drive/.shortcut-targets-by-id/19H3-omdRqkuz78Id2uV6htaMeqX7o4ou/davinci2automata
/content/drive/.shortcut-targets-by-id/19H3-omdRqkuz78Id2uV6htaMeqX7o4ou/davinci2automata


In [2]:
!pip install openai transformers
!pip install spaCy
!pip install termcolor ansi2html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00


In [13]:
#@title OpenAI credentials { run: "auto", display-mode: "form" }
import openai
ORGANIZATION_ID = "USE YOUR OPENAI ID" #@param {type:"string"}
OPENAI_API_KEY = "USE YOUR OPENAI KEY" #@param {type:"string"}
openai.organization = ORGANIZATION_ID # https://beta.openai.com/account/org-settings
openai.api_key = OPENAI_API_KEY # https://beta.openai.com/account/api-keys

In [4]:
import importlib
import termcolor
import functools
import numpy as np
import re

In [5]:
import openai
import spacy
from spacy import displacy

----
### GPT-**3**

In [6]:
import completion
importlib.reload(completion)
from completion import CompletionRequest, OneWordRequest, YesNoRequest, text2token

import structures
importlib.reload(structures)
from structures import Anchor, AnchorTree, bAnchor, pAnchor

#### Demo

In [7]:
params = dict(
  model="text-davinci-002",
  temperature=0.7,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
)
# see https://beta.openai.com/docs/api-reference/completions/create

##### Steps for "Cross the road"

In [8]:
import completion
importlib.reload(completion)
from completion import CompletionRequest, OneWordRequest, YesNoRequest, text2token

import structures
importlib.reload(structures)
from structures import Anchor, AnchorTree, bAnchor, pAnchor

class StepAnchor(bAnchor): pass

def extract_steps_from_text(text, steps=None):
  if steps is None: steps = AnchorTree(Anchor=StepAnchor)
  for line in text.splitlines():
    match = re.match(steps.Anchor._delim_regex(), line) # match lines STARTING with anchors
    if not match: continue
    steps[match.group()] = match.string[match.end():].strip()
  return steps

In [9]:
task = "Cross the road"
steps = AnchorTree(Anchor=StepAnchor)
steps[()] = task # root

logit_bias = {
    text2token(' ['): 4, # promote the use of "[x]"
    text2token(' proceed'): 4, # promote the use of "if ..., proceed to [x]"
    text2token(' go'): 4, text2token(' back'): 4, # promote the use of "if ..., go back to [x]"
    text2token(' If'): 4, # promote the use of "if ..."
    text2token('\n'): 4, # this helps having each step on one line
}
params = dict(
    model="text-davinci-002",
    max_tokens=300,
    # temperature=0.3, top_p=1,
    temperature=0., top_p=1,
    frequency_penalty=0, presence_penalty=0,
    logprobs=5, logit_bias=logit_bias,
)

steps

AnchorTree({StepAnchor(): 'Cross the road'}, Anchor=StepAnchor)

In [39]:
a = steps.Anchor() # root
request_lvl1 = request = CompletionRequest(
    prompt = f"Steps for: {steps[a]}\n\n{a.child(1)}",
    **params,
)

extract_steps_from_text(request.full_text(), steps)
steps

AnchorTree({StepAnchor(): 'Cross the road', StepAnchor(1): 'Look both ways before crossing the road.', StepAnchor(2): 'If there are no cars coming, proceed to cross the road.', StepAnchor(3): 'If there are cars coming, wait for them to pass before crossing the road.', StepAnchor(4): 'Once you have crossed the road, proceed to your destination.'}, Anchor=StepAnchor)

In [21]:
{
    **params,
    **params,
}

{'model': 'text-davinci-002',
 'max_tokens': 300,
 'temperature': 0.0,
 'top_p': 1,
 'frequency_penalty': 0,
 'presence_penalty': 0,
 'logprobs': 5,
 'logit_bias': {685: 4, 5120: 4, 467: 4, 736: 4, 1002: 4, 198: 4}}

In [47]:
# substeps

for i in range(1, 4):
    request_lvl2 = request = CompletionRequest(
        previous=request,
        prompt = f"Substeps for: {a.child(i)} {steps[a.child(i)]}\n\n{a.child(i).child(1)}",
        **{
          **params,
          'max_tokens': 500,
        },
        # dry_run=1,
        fail_on_partial=False,
    )

print(extract_steps_from_text(request.full_text()))

[1]   'Look both ways before crossing the road.'
[1.1] 'Face the direction you want to go.'
[1.2] 'Look to the left, then to the right, then back to the left again.'
[1.3] ''
[1.4] 'If there are cars coming, wait for them to pass before crossing the road.'
[2]   'If there are no cars coming, proceed to cross the road.'
[2.1] 'Walk at a steady pace across the road.'
[2.2] 'If a car does come, stop and wait for it to pass.'
[3]   'If there are cars coming, wait for them to pass before crossing the road.'
[3.1] 'If there is more than one car coming, wait for the first car to pass, then proceed to cross the road.'
[3.2] 'If there is only one car coming, wait for it to pass, then proceed to cross the road.'
